### Устанавливаем и запускаем Vertica

In [ ]:
pip install vertica-python

In [2]:
import os

In [3]:
os.system('docker run -d -p 5433:5433 jbfavre/vertica:latest')

125

### Подключаемся к БД

In [4]:
import vertica_python

In [5]:
connection_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
} 

### Проводим Миграции

In [3]:
with vertica_python.connect(**connection_info) as connection:
    cursor = connection.cursor()
    cursor.execute("""CREATE TABLE movie_view (
    id IDENTITY, user_uuid VARCHAR(256), movie_uuid VARCHAR(256), movie_progress VARCHAR(256), movie_length VARCHAR(256), event_time VARCHAR(256));
""")


### Заполняем БД на 1 000 000 записей

In [9]:
import time
import pandas
from pydantic import parse_obj_as, BaseModel
import datetime
from typing import List
import random
import uuid

In [10]:
movie_uuids = [str(uuid.uuid4()) for _ in range(1000)]
user_uuids = [str(uuid.uuid4()) for _ in range(2000)]

In [11]:
length = random.randint(600, 90000)
movies_gen = ((
    random.choice(user_uuids), random.choice(movie_uuids), str(random.randint(0, length)), str(length), str(datetime.datetime.now())) for _ in range(10_000_000)
)


In [ ]:
%%time
step = 10000
count = 1_000_000 // step
for _ in range(count):
    with vertica_python.connect(**connection_info) as connection:
        cursor = connection.cursor()
        cursor.executemany("""INSERT INTO movie_view (
                           user_uuid, movie_uuid, movie_progress, movie_length, event_time)
                           VALUES (?, ?, ?, ?, ?)""",
                           [next(movies_gen) for _ in range(step)],
                           use_prepared_statements=True) 


In [ ]:
with vertica_python.connect(**connection_info) as connection:
    cursor = connection.cursor()
    cursor.execute('SELECT COUNT(*) from movie_view')
    print(cursor.fetchall())

За час на одном потоке было загружено 60000 записей, дальнейшее тестирование считаю нецелесообразным в связи с тем что для нас скорость загрузки да